In [248]:
import heapq
from math import log2
from tabulate import tabulate

In [249]:
class ShannonFano:
    def __init__(self):
        self.codes = {}
    
    def encode(self, symbols):
        """Кодирование методом Шеннона-Фано"""
        # Сортируем символы по убыванию вероятностей
        sorted_symbols = sorted(symbols.items(), key=lambda x: x[1], reverse=True)
        self._build_code(sorted_symbols, "")
        return self.codes
    
    def _build_code(self, symbols, prefix) -> None:
        """Рекурсивное построение кода"""
        if len(symbols) == 1:
            self.codes[symbols[0][0]] = prefix
            return
        
        # Находим точку разделения
        total_prob = sum(prob for _, prob in symbols)
        half_prob = total_prob / 2
        
        current_prob = 0
        split_index = 0
        
        for i, (symbol, prob) in enumerate(symbols):
            current_prob += prob
            if current_prob >= half_prob:
                split_index = i + 1
                break
        
        # Рекурсивно строим коды для двух групп
        self._build_code(symbols[:split_index], prefix + "0")
        self._build_code(symbols[split_index:], prefix + "1")

In [250]:
class HuffmanNode:
    def __init__(self, symbol=None, prob=0):
        self.symbol = symbol
        self.prob = prob
        self.left = None
        self.right = None
    
    def __lt__(self, other):
        return self.prob < other.prob

In [251]:
class Huffman:
    def __init__(self):
        self.codes = {}
    
    def encode(self, symbols):
        """Кодирование методом Хаффмена"""
        # Создаем узлы для каждого символа
        heap = []
        for symbol, prob in symbols.items():
            node = HuffmanNode(symbol, prob)
            heapq.heappush(heap, node)
        
        # Строим дерево Хаффмена
        while len(heap) > 1:
            left = heapq.heappop(heap)
            right = heapq.heappop(heap)
            
            merged = HuffmanNode(None, left.prob + right.prob)
            merged.left = left
            merged.right = right
            
            heapq.heappush(heap, merged)
        
        # Генерируем коды
        root = heap[0]
        self._generate_codes(root, "")
        return self.codes
    
    def _generate_codes(self, node, code):
        """Рекурсивная генерация кодов"""
        if node is None:
            return
        
        if node.symbol is not None:
            self.codes[node.symbol] = code
            return
        
        self._generate_codes(node.left, code + "0")
        self._generate_codes(node.right, code + "1")

In [252]:
def calculate_metrics(symbols, codes):
    """Вычисление среднего числа символов в коде и избыточности"""
    # Средняя длина кода
    avg_length = 0
    for symbol, prob in symbols.items():
        code_length = len(codes[symbol])
        avg_length += prob * code_length
    
    # Энтропия
    entropy = 0
    for prob in symbols.values():
        if prob > 0:
            entropy -= prob * log2(prob)
    
    # Избыточность
    redundancy = avg_length - entropy
    
    return avg_length, entropy, redundancy

In [253]:
def get_symbols_with_probs(text: str):
    symbols = {}

    for i in text:
        symbols[i] = symbols.get(i, 0) + 1
    
    for k in symbols.keys():
        symbols[k] = symbols[k] / len(text)

    return symbols

In [254]:
def build_tree_from_codes(codes):
    """Строит дерево из словаря кодов {символ: двоичный_код}"""
    root = {}
    
    for symbol, code in codes.items():
        node = root
        for bit in code:
            if bit not in node:
                node[bit] = {}
            node = node[bit]
        node['symbol'] = symbol
    
    return root

In [255]:
def print_tree(tree, level=0):
    """Рекурсивно печатает дерево"""
    if 'symbol' in tree:
        print("  " * level + f"└── [{tree['symbol']}")
        return
    
    for bit, subtree in tree.items():
        print("  " * level + f"└── {bit}")
        print_tree(subtree, level + 1)

In [256]:
def start_programm(symbols_with_probs: dict[str, float]):
    print(f"Символы и вероятности:")
    for symbol, prob in symbols_with_probs.items():
        print(f"  {symbol}: {prob:.3f}")

    shannon_fano = ShannonFano()
    sf_codes = shannon_fano.encode(symbols_with_probs)
    sf_avg_length, sf_entropy, sf_redundancy = calculate_metrics(symbols_with_probs, sf_codes)

    huffman = Huffman()
    hf_codes = huffman.encode(symbols_with_probs)
    hf_avg_length, hf_entropy, hf_redundancy = calculate_metrics(symbols_with_probs, hf_codes)


    print("Кодирование символов методами Шеннона-Фано и Хаффмена")
    print("=" * 50)


    # Для кодов символов
    table_codes = []
    for symbol in symbols_with_probs:
        table_codes.append([symbol, symbols_with_probs[symbol], sf_codes[symbol], hf_codes[symbol]])

    print(tabulate(table_codes, 
                headers=['Символ', 'Вероятность', 'Шеннон-Фано', 'Хаффмен'],
                tablefmt='grid'))

    # Для метрик
    table_metrics = [
        ['Средняя длина', f"{sf_avg_length:.3f}", f"{hf_avg_length:.3f}"],
        ['Энтропия', f"{sf_entropy:.3f}", f"{hf_entropy:.3f}"],
        ['Избыточность', f"{sf_redundancy:.3f}", f"{hf_redundancy:.3f}"]
    ]

    print(tabulate(
        table_metrics,
        headers=['Метрика', 'Шеннон-Фано', 'Хаффмен'],
        tablefmt='grid'
    ))

    print("дерево Шеннон-Фано")
    print_tree(build_tree_from_codes(sf_codes))
    print("дерево Хаффмен")
    print_tree(build_tree_from_codes(hf_codes))


    

In [257]:
TEXT = "gvbncdhjbnchjadsbnjkcdsnjkdscn"

# TEXT = input("текст из которого будут взяты символы и их вероятность: ")

start_programm(get_symbols_with_probs(TEXT))

Символы и вероятности:
  g: 0.033
  v: 0.033
  b: 0.100
  n: 0.167
  c: 0.133
  d: 0.133
  h: 0.067
  j: 0.133
  a: 0.033
  s: 0.100
  k: 0.067
Кодирование символов методами Шеннона-Фано и Хаффмена
+----------+---------------+---------------+-----------+
| Символ   |   Вероятность |   Шеннон-Фано |   Хаффмен |
+==========+===============+===============+===========+
| g        |     0.0333333 |          1101 |     11010 |
+----------+---------------+---------------+-----------+
| v        |     0.0333333 |          1110 |     11011 |
+----------+---------------+---------------+-----------+
| b        |     0.1       |          1000 |       000 |
+----------+---------------+---------------+-----------+
| n        |     0.166667  |           000 |       111 |
+----------+---------------+---------------+-----------+
| c        |     0.133333  |           001 |       011 |
+----------+---------------+---------------+-----------+
| d        |     0.133333  |           010 |       100 |
+---

In [258]:
symbols_with_probs = dict()

symbols_with_probs = {
    "a": 0.5,
    "b": 0.2,
    "c": 0.3,
}

# n = int(input("Введите количество символов: "))
# for i in range(n):
#     symbol = input(f"Символ {i+1}: ").strip()
#     prob = float(input(f"Вероятность для '{symbol}': "))
#     symbols_with_probs[symbol] = prob

start_programm(symbols_with_probs)

Символы и вероятности:
  a: 0.500
  b: 0.200
  c: 0.300
Кодирование символов методами Шеннона-Фано и Хаффмена
+----------+---------------+---------------+-----------+
| Символ   |   Вероятность |   Шеннон-Фано |   Хаффмен |
+==========+===============+===============+===========+
| a        |           0.5 |             0 |         0 |
+----------+---------------+---------------+-----------+
| b        |           0.2 |            11 |        10 |
+----------+---------------+---------------+-----------+
| c        |           0.3 |            10 |        11 |
+----------+---------------+---------------+-----------+
+---------------+---------------+-----------+
| Метрика       |   Шеннон-Фано |   Хаффмен |
+===============+===============+===========+
| Средняя длина |         1.5   |     1.5   |
+---------------+---------------+-----------+
| Энтропия      |         1.485 |     1.485 |
+---------------+---------------+-----------+
| Избыточность  |         0.015 |     0.015 |
+--------